In [1]:
import tensorflow as tf
import pandas as pd, numpy as np
import sklearn

In [2]:
X = pd.read_csv('../Data_manipulation/experts_dataset/x.tsv', sep = '\t')
y = pd.read_csv('../Data_manipulation/experts_dataset/y.tsv', sep = '\t')

In [3]:
X.head()

,display_id,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,1,0.034416,0.034416,0.039079,0.98829,0.85059,0.086827,0.054094,1.0,0.523395,...,0,0,0,0,0,0,0,0,0,0
1,2,0.034416,0.034416,0.039079,0.98829,0.85059,0.086827,0.054094,1.0,0.523395,...,0,0,0,0,0,0,0,0,0,0
2,3,0.034416,0.034416,0.039079,0.98829,0.85059,0.086827,0.054094,1.0,0.523395,...,0,0,0,1,0,0,0,0,0,0
3,4,0.034416,0.034416,0.039079,0.98829,0.85059,0.086827,0.054094,1.0,0.523395,...,0,0,0,0,0,0,0,0,0,0
4,5,0.034416,0.034416,0.039079,0.98829,0.85059,0.086827,0.054094,1.0,0.523395,...,0,0,0,0,0,0,0,0,0,0


In [4]:
y.head()

,parent_display_id,label
0,1,1
1,2,5
2,4,1
3,4,5
4,4,5


In [5]:
j = pd.merge(X, y, left_on = 'display_id', right_on = 'parent_display_id')
X = j.drop(['label', 'display_id', 'parent_display_id'], axis = 1)
y = j.label

In [6]:
print(X.shape)
X.head()

(503, 51)


,1,2,3,4,5,6,7,8,9,10,...,42,43,44,45,46,47,48,49,50,51
0,0.034416,0.034416,0.039079,0.98829,0.85059,0.086827,0.054094,1.0,0.523395,0.481127,...,0,0,0,0,0,0,0,0,0,0
1,0.034416,0.034416,0.039079,0.98829,0.85059,0.086827,0.054094,1.0,0.523395,0.481127,...,0,0,0,0,0,0,0,0,0,0
2,0.034416,0.034416,0.039079,0.98829,0.85059,0.086827,0.054094,1.0,0.523395,0.481127,...,0,0,0,0,0,0,0,0,0,0
3,0.034416,0.034416,0.039079,0.98829,0.85059,0.086827,0.054094,1.0,0.523395,0.481127,...,0,0,0,0,0,0,0,0,0,0
4,0.034416,0.034416,0.039079,0.98829,0.85059,0.086827,0.054094,1.0,0.523395,0.481127,...,0,0,0,0,0,0,0,0,0,0


In [7]:
y.head()

0    1
1    5
2    1
3    5
4    5
Name: label, dtype: int64

In [8]:
y.value_counts()

5     141
4     107
2      92
11     67
9      38
1      34
10     15
0       5
6       2
8       1
7       1
Name: label, dtype: int64

In [9]:
OP_NUMBER = 12
STATE_INPUT_SIZE = 51
HIDDEN_SIZE = 30


In [10]:
assert(STATE_INPUT_SIZE == X.shape[1])
assert(OP_NUMBER == y.max() + 1)

In [11]:
#Initializing 
tf.reset_default_graph()

W1 = tf.get_variable(shape=[HIDDEN_SIZE,STATE_INPUT_SIZE],name='w1',
                      initializer=tf.contrib.layers.xavier_initializer())
W2 = tf.get_variable(shape=[HIDDEN_SIZE,HIDDEN_SIZE],name='w2',
                      initializer=tf.contrib.layers.xavier_initializer())
W3 = tf.get_variable(shape=[OP_NUMBER,HIDDEN_SIZE],name='w3',
                      initializer=tf.contrib.layers.xavier_initializer())

b1 = tf.get_variable(shape=[HIDDEN_SIZE,1],name='b1',
                      initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable(shape=[HIDDEN_SIZE,1],name='b2',
                      initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.get_variable(shape=[OP_NUMBER,1],name='b3',
                      initializer=tf.contrib.layers.xavier_initializer())

#Layers:
x = tf.placeholder(tf.float32, shape=[STATE_INPUT_SIZE,None],name='x')
h1 = tf.tanh(tf.matmul(W1,x) + b1)
h2 = tf.tanh(tf.matmul(W2,h1) + b2)
y_logits = tf.matmul(W3,h2) + b3
y = tf.nn.softmax(y_logits,dim=0)



In [12]:
y_label = tf.placeholder(tf.float32,[OP_NUMBER, None], name="y_label")


In [13]:
#ADAM Optimizer hyper-parameters:
LEARNING_RATE = 0.01
B1 = 0.8
B2= 0.999
EPSILON=1e-6


In [14]:
ce1 = tf.nn.softmax_cross_entropy_with_logits(logits=y_logits, labels=y_label, dim=0)
adam = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE,beta1=B1,beta2=B2,epsilon=EPSILON)
train_step = adam.minimize(ce1)

In [15]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [16]:
BATCH_SIZE = 10
BATCHES = 5000

def feed_dict_from_df(j):
    xs, ys = [], []
    X = j.drop(['label', 'display_id', 'parent_display_id'], axis = 1)
    y = j.label
    for i in range(bdf.shape[0]):
        xs.append(X.iloc[i].values)
        y_hot = [0]*OP_NUMBER
        y_hot[y.iloc[i]] = 1
        ys.append(y_hot)
    return np.vstack(xs).T, np.vstack(ys).T
        
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for batch in range(BATCHES):
    bdf = j.sample(BATCH_SIZE)
    xs, ys = feed_dict_from_df(bdf)
    sess.run(train_step, feed_dict={x: xs, y_label: ys})
    if batch % (BATCHES/10) == 0:
        train_accuracy = accuracy.eval(feed_dict={x: xs, y_label: ys}, session = sess)
        print('batch %d, training accuracy %g' % (batch, train_accuracy))
xs, ys = feed_dict_from_df(j)
test_accuracy = accuracy.eval(feed_dict={x: xs, y_label: ys}, session = sess)
print('overall accuracy %g' % (test_accuracy))


batch 0, training accuracy 0.0833333
batch 500, training accuracy 0.583333
batch 1000, training accuracy 0.666667
batch 1500, training accuracy 0.833333
batch 2000, training accuracy 0.666667
batch 2500, training accuracy 0.75
batch 3000, training accuracy 0.666667
batch 3500, training accuracy 0.666667
batch 4000, training accuracy 0.666667
batch 4500, training accuracy 0.666667
overall accuracy 0.833333


In [17]:
saver = tf.train.Saver()
saver.save(sess, '../models/ianna-nn-supervised')

'../models/ianna-nn-supervised'

In [18]:
sess.close()